In [1]:
import ipywidgets as widgets
import torch
import PIL
from IPython.display import display
import io
import numpy as np
import bqplot
from matplotlib import pyplot as plt
import matplotlib as mpl

from ppuu.eval_mpc_visualizer import EvalVisualizer

class WidgetEvalVisualizer(EvalVisualizer, widgets.VBox):
    def __init__(self, output_dir=None):
        EvalVisualizer.__init__(self, output_dir, notebook=True)
        widgets.VBox.__init__(self, [self.costs_plot_output])
        
    def setup_mpl(self):
        # in the other class we'd do use non-visual backend
        pass
    
    def show_mpl(self):
        plt.show()

visualizer = WidgetEvalVisualizer()
display(visualizer)

WidgetEvalVisualizer(children=(Output(),))

In [2]:
import ppuu.eval_mpc 
from ppuu.costs.policy_costs_km import PolicyCostKMTaper
import imp
imp.reload(ppuu.eval_mpc)

config = ppuu.eval_mpc.EvalMPCConfig.parse_from_flat_dict(
    dict(
    lambda_p=7.97, 
    lambda_o=3.46,#2.87,
    lambda_l=1.08,#3.05,
    lr=0.4814,
    u_reg=0.0,
    lambda_a=0.001,
    lambda_j=0.0,
    lambda_r=1.0,
    lambda_d=1,
    lambda_j_mpc=0.001, # 0.1,
    lambda_s=0, # 0.1,
    skip_contours=True,
    masks_power_x=13,
    masks_power_y=13,
    timestep=0.1,
    safe_factor=3,
    unfold_len=30,
    n_iter=25,
    dataset="/home/us441/nvidia-collab/vlad/traffic-data-5/state-action-cost/data_i80_v0/",
    forward_model_path="/home/us441/nvidia-collab/vlad/results/fm/km_no_action/fm_km_no_action_64/seed=42/checkpoints/last.ckpt",
    visualizer=visualizer,
    num_processes=0,
    test_size_cap=10,
#     dataset_one_episode=245,
    optimizer='CE',
    ce_gd=False,
    # optimizer='LBFGS',
    lbfgs_limit=3.0,
    lbfgs_history_size=2,
    lambda_action_log_barrier=1000,
    fm_unfold_samples=2,
    fm_unfold_samples_agg='max',
    fm_unfold_variance=1.0,
    clip_actions=5.0,
    planning_freq=3,
    batch_size=50,
    mask_coeff=100.0,
    update_ref_period=100.0,
    pass_gt_future=True,
    output_dir=None,
    dataset_partition='test',
    build_overlay=False,
    build_cost_profile_and_traj=False,
    unfolding_agg='max',
    plan_size=5,
    target_speed=30,
    )
)
# policy = ppuu.eval_mpc.main(config)

In [ ]:
# import cProfile
# cProfile.run('ppuu.eval_mpc.main(config)', 'restats')
ppuu.eval_mpc.main(config)

pygame 2.0.0.dev6 (SDL 2.0.10, python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
final actions for 1 are tensor([[ 1.5507e+01, -3.0417e-04]], device='cuda:0')
final actions for 2 are tensor([[ 1.5350e+01, -2.0976e-04]], device='cuda:0')
final actions for 3 are tensor([[-4.8354e+00,  1.5267e-04]], device='cuda:0')
final actions for 4 are tensor([[1.1655e+01, 5.3426e-04]], device='cuda:0')
final actions for 5 are tensor([[ 1.5429e+01, -2.1446e-03]], device='cuda:0')
final actions for 6 are tensor([[1.5325e+01, 2.0899e-03]], device='cuda:0')
final actions for 7 are tensor([[ 7.2054e+00, -3.2843e-04]], device='cuda:0')
final actions for 8 are tensor([[1.5516e+01, 4.8815e-04]], device='cuda:0')
final actions for 9 are tensor([[-0.2926,  0.0010]], device='cuda:0')
final actions for 10 are tensor([[ 9.9751e+00, -9.1369e-04]], device='cuda:0')
final actions for 11 are tensor([[1.5072e+01, 1.6578e-03]], device='cuda:0')
final actions for 12 are tensor([[ 2.74

## %debug

In [4]:
class EpisodeReview(widgets.VBox):
    def __init__(self, frames):
        self.frames = frames
        self.episode_play = widgets.Play(
            value=0,
            min=0,
            max=len(frames),
            step=1,
            description="Press play",
            disabled=False,
            interval=10,
        )
        self.update_interval_slider = widgets.IntSlider(
            min=1, max=300, value=30,
        )

        self.episode_image = widgets.Image(
            format="png", width=600, height=600,
        )
        self.episode_slider = widgets.IntSlider()
        widgets.jslink(
            (self.episode_play, "value"), (self.episode_slider, "value")
        )
        widgets.jslink(
            (self.episode_play, "max"), (self.episode_slider, "max")
        )
        widgets.jslink(
            (self.episode_play, "min"), (self.episode_slider, "min")
        )
        
        widgets.jslink(
            (self.update_interval_slider, "value"),
            (self.episode_play, "interval"),
        )

        
        def episode_slider_callback(change):
            if change.name == "value" and change.new is not None:
                self.episode_image.value = self.frames[change.new]

        self.episode_slider.observe(episode_slider_callback, type="change")
        
        super().__init__(
            [self.episode_play, self.episode_slider, self.update_interval_slider, self.episode_image]
        )

In [6]:
episode_view = EpisodeReview(visualizer.images_history[0])
display(episode_view)

IndexError: list index out of range

In [ ]:
import pstats
s = pstats.Stats('restats')